# 데이터 대회 - 기본 분석 구조
## 1. 라이브러리 및 설정

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 시각화 스타일 설정
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print('라이브러리 로드 완료!')

## 2. 데이터 로드

In [ ]:
# Train 데이터 로드
train = pd.read_csv('train.csv')
print(f'Train 데이터 shape: {train.shape}')
print(f'Train 데이터 크기: {train.shape[0]:,}행 x {train.shape[1]:,}열\n')

# Test 데이터 로드
test = pd.read_csv('test.csv')
print(f'Test 데이터 shape: {test.shape}')
print(f'Test 데이터 크기: {test.shape[0]:,}행 x {test.shape[1]:,}열')

In [ ]:
# 데이터 미리보기
print('=== Train 데이터 상위 5행 ===')
display(train.head())

print('\n=== Test 데이터 상위 5행 ===')
display(test.head())

In [ ]:
# 데이터 타입 및 기본 정보 확인
print('=== Train 데이터 정보 ===')
train.info()

print('\n=== Test 데이터 정보 ===')
test.info()

## 3. 결측치(Null) 확인 및 처리

In [ ]:
# Train 데이터 결측치 확인
print('=== Train 데이터 결측치 현황 ===')
train_null = train.isnull().sum()
train_null_pct = (train.isnull().sum() / len(train)) * 100

null_df_train = pd.DataFrame({
    '결측치 개수': train_null,
    '결측치 비율(%)': train_null_pct
})
null_df_train = null_df_train[null_df_train['결측치 개수'] > 0].sort_values('결측치 개수', ascending=False)

if len(null_df_train) > 0:
    display(null_df_train)
else:
    print('Train 데이터에 결측치가 없습니다!')

In [ ]:
# Test 데이터 결측치 확인
print('=== Test 데이터 결측치 현황 ===')
test_null = test.isnull().sum()
test_null_pct = (test.isnull().sum() / len(test)) * 100

null_df_test = pd.DataFrame({
    '결측치 개수': test_null,
    '결측치 비율(%)': test_null_pct
})
null_df_test = null_df_test[null_df_test['결측치 개수'] > 0].sort_values('결측치 개수', ascending=False)

if len(null_df_test) > 0:
    display(null_df_test)
else:
    print('Test 데이터에 결측치가 없습니다!')

In [ ]:
# 결측치 처리 함수 정의
def handle_missing_values(df, method='mean'):
    """
    결측치 처리 함수
    
    Parameters:
    - df: DataFrame
    - method: 'mean' (평균), 'median' (중앙값), 'mode' (최빈값), 'drop' (삭제), 'ffill' (전방채움), 'bfill' (후방채움)
    
    Returns:
    - 결측치가 처리된 DataFrame
    """
    df_copy = df.copy()
    
    if method == 'mean':
        # 수치형 컬럼만 평균으로 채우기
        numeric_cols = df_copy.select_dtypes(include=[np.number]).columns
        df_copy[numeric_cols] = df_copy[numeric_cols].fillna(df_copy[numeric_cols].mean())
        print(f'수치형 컬럼 {len(numeric_cols)}개를 평균값으로 채웠습니다.')
        
    elif method == 'median':
        # 수치형 컬럼만 중앙값으로 채우기
        numeric_cols = df_copy.select_dtypes(include=[np.number]).columns
        df_copy[numeric_cols] = df_copy[numeric_cols].fillna(df_copy[numeric_cols].median())
        print(f'수치형 컬럼 {len(numeric_cols)}개를 중앙값으로 채웠습니다.')
        
    elif method == 'mode':
        # 모든 컬럼을 최빈값으로 채우기
        for col in df_copy.columns:
            if df_copy[col].isnull().sum() > 0:
                df_copy[col].fillna(df_copy[col].mode()[0], inplace=True)
        print('모든 컬럼을 최빈값으로 채웠습니다.')
        
    elif method == 'drop':
        # 결측치가 있는 행 삭제
        before_len = len(df_copy)
        df_copy = df_copy.dropna()
        print(f'{before_len - len(df_copy)}개 행을 삭제했습니다.')
        
    elif method == 'ffill':
        # 전방 채우기
        df_copy = df_copy.fillna(method='ffill')
        print('전방 채우기(forward fill)를 수행했습니다.')
        
    elif method == 'bfill':
        # 후방 채우기
        df_copy = df_copy.fillna(method='bfill')
        print('후방 채우기(backward fill)를 수행했습니다.')
    
    return df_copy

In [ ]:
# 결측치 처리 적용 (필요시 method 변경)
# method 옵션: 'mean', 'median', 'mode', 'drop', 'ffill', 'bfill'

print('=== Train 데이터 결측치 처리 ===')
train_processed = handle_missing_values(train, method='mean')

print('\n=== Test 데이터 결측치 처리 ===')
test_processed = handle_missing_values(test, method='mean')

# 처리 후 결측치 확인
print(f'\nTrain 결측치 총 개수: {train_processed.isnull().sum().sum()}')
print(f'Test 결측치 총 개수: {test_processed.isnull().sum().sum()}')

## 4. 데이터 구조 파악 (x, y, p 변수)

In [ ]:
# x, y, p 컬럼 분리
x_cols = [col for col in train_processed.columns if col.startswith('x')]
y_cols = [col for col in train_processed.columns if col.startswith('y')]
p_cols = [col for col in train_processed.columns if col.startswith('p')]

print(f'x 변수 개수: {len(x_cols)}개 (x0 ~ x255)')
print(f'y 변수 개수: {len(y_cols)}개')
print(f'p 변수 개수: {len(p_cols)}개')
print(f'\n총 변수 개수: {len(train_processed.columns)}개')

In [ ]:
# 기본 통계량 확인
print('=== X 변수 통계량 (일부) ===')
display(train_processed[x_cols[:5]].describe())

print('\n=== Y 변수 통계량 (일부) ===')
display(train_processed[y_cols[:5]].describe())

print('\n=== P 변수 통계량 (일부) ===')
display(train_processed[p_cols[:5]].describe())

## 5. 3D 시각화 (샘플 데이터)

In [ ]:
# 3D 시각화를 위한 샘플 행 선택
# 각 샘플(행)에서 x0~x255, y0~y255, p0~p255를 3D 공간에 표시

# 랜덤하게 몇 개 샘플 선택 (너무 많으면 복잡하므로)
np.random.seed(42)
num_samples = 3  # 3개 샘플만 선택
sample_indices = np.random.choice(len(train_processed), size=min(num_samples, len(train_processed)), replace=False)

print(f'선택된 샘플 인덱스: {sample_indices}')
print(f'각 샘플마다 {len(x_cols)}개의 3D 점을 표시합니다.')
print(f'- x축: x0~x{len(x_cols)-1} 값')
print(f'- y축: y0~y{len(y_cols)-1} 값')  
print(f'- z축: p0~p{len(p_cols)-1} 값')

In [ ]:
# 3D 시각화 함수 - 하나의 샘플에서 모든 (x, y, p) 점들을 표시
def plot_sample_3d(data, sample_idx, x_cols, y_cols, p_cols):
    """
    하나의 샘플(행)에 대해 x, y, p 값들을 3D 공간에 표시
    
    Parameters:
    - data: DataFrame
    - sample_idx: 샘플 인덱스
    - x_cols: x 컬럼 리스트
    - y_cols: y 컬럼 리스트
    - p_cols: p 컬럼 리스트
    """
    # 해당 샘플의 x, y, p 값 추출
    sample = data.iloc[sample_idx]
    
    x_values = sample[x_cols].values
    y_values = sample[y_cols].values
    p_values = sample[p_cols].values
    
    # 길이가 다를 수 있으므로 최소 길이로 맞춤
    min_len = min(len(x_values), len(y_values), len(p_values))
    x_values = x_values[:min_len]
    y_values = y_values[:min_len]
    p_values = p_values[:min_len]
    
    # 3D plot
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # 산점도 - p값에 따라 색상 변화
    scatter = ax.scatter(x_values, y_values, p_values,
                        c=p_values, 
                        cmap='viridis',
                        alpha=0.6,
                        s=20,
                        edgecolors='w',
                        linewidth=0.5)
    
    ax.set_xlabel('X 값 (x0~x{})'.format(len(x_cols)-1), fontsize=12, labelpad=10)
    ax.set_ylabel('Y 값 (y0~y{})'.format(len(y_cols)-1), fontsize=12, labelpad=10)
    ax.set_zlabel('P 값 (p0~p{})'.format(len(p_cols)-1), fontsize=12, labelpad=10)
    ax.set_title(f'샘플 #{sample_idx} - (X, Y, P) 3D 분포\n총 {min_len}개 점', 
                 fontsize=14, pad=20)
    
    # 컬러바 추가
    cbar = plt.colorbar(scatter, ax=ax, shrink=0.5, aspect=5, pad=0.1)
    cbar.set_label('P 값', rotation=270, labelpad=15)
    
    # 그리드 추가
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 통계 정보 출력
    print(f'샘플 #{sample_idx} 통계:')
    print(f'  X 범위: [{x_values.min():.3f}, {x_values.max():.3f}]')
    print(f'  Y 범위: [{y_values.min():.3f}, {y_values.max():.3f}]')
    print(f'  P 범위: [{p_values.min():.3f}, {p_values.max():.3f}]')
    print()

In [ ]:
# 첫 번째 샘플 시각화
print('=== 첫 번째 샘플 3D 시각화 ===')
plot_sample_3d(train_processed, sample_indices[0], x_cols, y_cols, p_cols)

In [ ]:
# 두 번째 샘플 시각화
print('=== 두 번째 샘플 3D 시각화 ===')
plot_sample_3d(train_processed, sample_indices[1], x_cols, y_cols, p_cols)

In [ ]:
# 세 번째 샘플 시각화
print('=== 세 번째 샘플 3D 시각화 ===')
plot_sample_3d(train_processed, sample_indices[2], x_cols, y_cols, p_cols)

In [ ]:
# 여러 샘플을 한 번에 비교 (서브플롯)
fig = plt.figure(figsize=(20, 6))

for i, sample_idx in enumerate(sample_indices):
    # 샘플 데이터 추출
    sample = train_processed.iloc[sample_idx]
    x_values = sample[x_cols].values
    y_values = sample[y_cols].values
    p_values = sample[p_cols].values
    
    # 길이 맞추기
    min_len = min(len(x_values), len(y_values), len(p_values))
    x_values = x_values[:min_len]
    y_values = y_values[:min_len]
    p_values = p_values[:min_len]
    
    # 서브플롯 생성
    ax = fig.add_subplot(1, 3, i+1, projection='3d')
    
    scatter = ax.scatter(x_values, y_values, p_values,
                        c=p_values, 
                        cmap=['viridis', 'plasma', 'coolwarm'][i],
                        alpha=0.6,
                        s=15)
    
    ax.set_xlabel('X 값', fontsize=10)
    ax.set_ylabel('Y 값', fontsize=10)
    ax.set_zlabel('P 값', fontsize=10)
    ax.set_title(f'샘플 #{sample_idx}', fontsize=12, pad=10)
    
    # 컬러바
    cbar = plt.colorbar(scatter, ax=ax, shrink=0.6, aspect=5)
    cbar.set_label('P', rotation=270, labelpad=10, fontsize=9)

plt.suptitle('3개 샘플의 (X, Y, P) 3D 분포 비교', fontsize=14, y=0.98)
plt.tight_layout()
plt.show()

print('3개 샘플을 동시에 비교했습니다!')

### 추가 시각화 옵션

In [ ]:
# Surface plot으로 시각화 (한 샘플의 X-Y 평면에 P를 높이로 표시)
# P값의 개수에 따라 적절한 그리드 생성

def plot_sample_3d_surface(data, sample_idx, x_cols, y_cols, p_cols):
    """
    Surface plot 스타일로 샘플 시각화
    """
    sample = train_processed.iloc[sample_idx]
    
    x_values = sample[x_cols].values
    y_values = sample[y_cols].values
    p_values = sample[p_cols].values
    
    min_len = min(len(x_values), len(y_values), len(p_values))
    x_values = x_values[:min_len]
    y_values = y_values[:min_len]
    p_values = p_values[:min_len]
    
    fig = plt.figure(figsize=(16, 6))
    
    # 왼쪽: Scatter plot
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    scatter = ax1.scatter(x_values, y_values, p_values,
                         c=p_values, cmap='viridis', alpha=0.6, s=20)
    ax1.set_xlabel('X 값')
    ax1.set_ylabel('Y 값')
    ax1.set_zlabel('P 값')
    ax1.set_title('Scatter Plot')
    plt.colorbar(scatter, ax=ax1, shrink=0.5)
    
    # 오른쪽: Line plot (인덱스에 따른 변화)
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    indices = np.arange(min_len)
    ax2.plot(x_values, y_values, p_values, 
            c='blue', alpha=0.5, linewidth=1)
    scatter2 = ax2.scatter(x_values, y_values, p_values,
                          c=p_values, cmap='plasma', s=15, alpha=0.8)
    ax2.set_xlabel('X 값')
    ax2.set_ylabel('Y 값')
    ax2.set_zlabel('P 값')
    ax2.set_title('Connected Plot (순서대로 연결)')
    plt.colorbar(scatter2, ax=ax2, shrink=0.5)
    
    plt.suptitle(f'샘플 #{sample_idx} - 다양한 시각화 방식', fontsize=14, y=0.98)
    plt.tight_layout()
    plt.show()

# 첫 번째 샘플로 테스트
print('=== 다양한 시각화 방식 비교 ===')
plot_sample_3d_surface(train_processed, sample_indices[0], x_cols, y_cols, p_cols)

In [ ]:
# 2D 투영 시각화 (X-Y, X-P, Y-P 평면)
def plot_sample_2d_projections(data, sample_idx, x_cols, y_cols, p_cols):
    """
    3D 데이터를 2D 평면에 투영하여 시각화
    """
    sample = data.iloc[sample_idx]
    
    x_values = sample[x_cols].values
    y_values = sample[y_cols].values
    p_values = sample[p_cols].values
    
    min_len = min(len(x_values), len(y_values), len(p_values))
    x_values = x_values[:min_len]
    y_values = y_values[:min_len]
    p_values = p_values[:min_len]
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # X-Y 평면 (P를 색상으로)
    scatter1 = axes[0].scatter(x_values, y_values, c=p_values, 
                               cmap='viridis', alpha=0.6, s=20)
    axes[0].set_xlabel('X 값')
    axes[0].set_ylabel('Y 값')
    axes[0].set_title('X-Y 평면 투영 (색상=P)')
    axes[0].grid(True, alpha=0.3)
    plt.colorbar(scatter1, ax=axes[0])
    
    # X-P 평면 (Y를 색상으로)
    scatter2 = axes[1].scatter(x_values, p_values, c=y_values,
                               cmap='plasma', alpha=0.6, s=20)
    axes[1].set_xlabel('X 값')
    axes[1].set_ylabel('P 값')
    axes[1].set_title('X-P 평면 투영 (색상=Y)')
    axes[1].grid(True, alpha=0.3)
    plt.colorbar(scatter2, ax=axes[1])
    
    # Y-P 평면 (X를 색상으로)
    scatter3 = axes[2].scatter(y_values, p_values, c=x_values,
                               cmap='coolwarm', alpha=0.6, s=20)
    axes[2].set_xlabel('Y 값')
    axes[2].set_ylabel('P 값')
    axes[2].set_title('Y-P 평면 투영 (색상=X)')
    axes[2].grid(True, alpha=0.3)
    plt.colorbar(scatter3, ax=axes[2])
    
    plt.suptitle(f'샘플 #{sample_idx} - 2D 평면 투영', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

# 첫 번째 샘플로 테스트
print('=== 2D 평면 투영 시각화 ===')
plot_sample_2d_projections(train_processed, sample_indices[0], x_cols, y_cols, p_cols)

## 6. 추가 분석을 위한 준비
### 여기서부터 Feature Engineering과 모델링을 진행하세요!

In [ ]:
# 처리된 데이터 저장 (선택사항)
# train_processed.to_csv('train_processed.csv', index=False)
# test_processed.to_csv('test_processed.csv', index=False)
# print('처리된 데이터를 저장했습니다!')

print('\n=== 데이터 준비 완료 ===')
print(f'Train 데이터: {train_processed.shape}')
print(f'Test 데이터: {test_processed.shape}')
print('\n이제 Feature Engineering과 모델링을 진행하세요!')

---
## Feature Engineering 섹션 (직접 작성)
여기에 Feature Engineering 코드를 작성하세요

In [ ]:
# Feature Engineering 코드
# TODO: 여기에 Feature Engineering 코드 작성


---
## 모델링 섹션 (직접 작성)
여기에 모델 학습 및 예측 코드를 작성하세요

In [ ]:
# 모델링 코드
# TODO: 여기에 모델링 코드 작성
